In [1]:
import os
from os import path
import re
import json
import time
import math

from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import spacy
import mysql.connector

DIRETORIO_PDF = 'BauruPDF/'
DIRETORIO_TXT = 'BauruTXT/'
DIRETORIO_DOC = 'BauruDOC/'
DIRETORIO_PAR = 'BauruPAR/'
DIRETORIO_JSON = 'BauruJSON/'

sql_config = {
    'user': 'root',
    'password': 'see-cret',
    'host': 'localhost',
    'database': 'tcc'
}

In [2]:
tokenizador = re.compile('\w+')

def monta_saco_ngram(texto, n = 2, ignora_digito = True):
    def monta_chave(tokens_anteriores, token_atual):
        chave = ''
        
        for token in tokens_anteriores:
            chave += token + '_'
        chave += token_atual
        
        return chave
    
    def atualiza_tokens(tokens_anteriores, token_atual):
        if n == 1:
            return
        else:
            i = 0
            while i < n - 2:
                tokens_anteriores[i] = tokens_anteriores[i + 1]
                i += 1
            tokens_anteriores[i] = token_atual
            return
    
    saco_ngram = dict()

    
    if ignora_digito:
        conteudo = re.sub('\d', ' ', texto.lower())
    else:
        conteudo = texto.lower()
        
    tokens = tokenizador.findall(conteudo)

    tokens_anteriores = []
    for i in range(n - 1):
        tokens_anteriores.append('')
    
    for token in tokens:
        chave = monta_chave(tokens_anteriores, token)
        if chave not in saco_ngram:
            saco_ngram[chave] = 1
        else:
            saco_ngram[chave] += 1
        atualiza_tokens(tokens_anteriores, token)

    return saco_ngram

def atualiza_saco_corpus(saco_corpus, saco):
    for token in saco:
        if token not in saco_corpus:
            saco_corpus[token] = saco[token]
        else:
            saco_corpus[token] += saco[token]

def atualiza_idf_corpus(saco_corpus, saco):
    for token in saco:
        if token not in saco_corpus:
            saco_corpus[token] = 1
        else:
            saco_corpus[token] += 1

def calcula_cosseno_sacos(saco_a, saco_b):
    produto_escalar = 0.0
    
    soma_a = 0.0
    soma_b = 0.0
    
    for k in saco_a:
        if k in saco_b:
            produto_escalar += saco_a[k] * saco_b[k]
        soma_a += math.pow(saco_a[k], 2)
    
    for k in saco_b:
        soma_b += math.pow(saco_b[k], 2)
    
    magnitude = math.sqrt(soma_a) * math.sqrt(soma_b)
    
    if abs(magnitude) < 1e-3:
        similaridade = 0
    else:
        similaridade = produto_escalar / magnitude
    
    return similaridade

def calcula_jaccard_sacos(saco_a, saco_b):
    intersec = 0
    
    for key in saco_a:
        if key in saco_b:
            intersec += 1
    uniao = len(saco_a) + len(saco_b) - intersec
    if uniao == 0:
        return 0
    return intersec/uniao

def calcula_dissimilaridade_strings(string_a, string_b, n = 2):
    saco_a = monta_saco_ngram(string_a, n = n, ignora_digito = False)
    saco_b = monta_saco_ngram(string_b, n = n, ignora_digito = False)
    
    tot_a = 0
    excl_a = 0
    ambos = 0
    excl_b = 0
    tot_b = 0
    
    for chave in saco_a:
        tot_a += saco_a[chave]
        
        if chave in saco_b:
            tot_b += saco_b[chave]
            if saco_a[chave] < saco_b[chave]:
                ambos += saco_a[chave]
                saco_b[chave] = saco_b[chave] - saco_a[chave]
                excl_b += saco_b[chave]
            else:
                ambos += saco_b[chave]
                saco_a[chave] = saco_a[chave] - saco_b[chave]
                excl_a += saco_a[chave]
            saco_b[chave] = 0
            saco_a[chave] = 0
            
        else:
            excl_a += saco_a[chave]
            saco_a[chave] = 0
            
    for chave in saco_b:
        tot_b += saco_b[chave]
        excl_b += saco_b[chave]
        saco_b[chave] = 0
    
#     print(tot_a, excl_a, ambos, excl_b, tot_b)

    if tot_a == 0 or tot_b == 0:
        return 1
    
    return (excl_a * excl_b) / (tot_a * tot_b)
#     return ambos / (1 + excl_a * excl_b)
            

In [7]:
### CÁLCULO DO IDF

pars_bag = {}
par_bag = {}

with mysql.connector.connect(**sql_config) as connection:
    cursor = connection.cursor()
    
    cursor.execute('SELECT conteudo FROM paragrafo')
        
    for conteudo, in tqdm(cursor):
        par_bag = dict()

        tokens = tokenizador.findall(re.sub('\d', ' ', conteudo.lower()))
        for token in tokens:
            par_bag[token] = True
            
        for token in par_bag:
            if token not in pars_bag:
                pars_bag[token] = 1
            else:
                pars_bag[token] += 1
    pars_n = cursor.rowcount

print(pars_n, 'paragrafos lidos')
print(len(pars_bag), 'tokens')

1987212it [01:09, 28688.49it/s]

1987212 paragrafos lidos
115679 tokens


In [18]:
### CÁLCULO DO IDF
pars_bag = {}
par_bag = {}

with mysql.connector.connect(**sql_config) as connection:
    cursor = connection.cursor()
    
    cursor.execute('SELECT conteudo FROM paragrafo')
        
    for conteudo, in tqdm(cursor):
        par_bag = monta_saco_ngram(conteudo, n = 1)
            
        for token in par_bag:
            if token not in pars_bag:
                pars_bag[token] = 1
            else:
                pars_bag[token] += 1
    pars_n = cursor.rowcount

print(pars_n, 'paragrafos lidos')
print(len(pars_bag), 'tokens')

1987212it [01:31, 21643.27it/s]

1987212 paragrafos lidos
115679 tokens


In [22]:
### CÁLCULO DO IDF
pars_bag = {}
par_bag = {}

with mysql.connector.connect(**sql_config) as connection:
    cursor = connection.cursor()
    
    cursor.execute('SELECT conteudo FROM paragrafo')
        
    for conteudo, in tqdm(cursor):
        par_bag = monta_saco_ngram(conteudo, n = 1)
            
        for token in par_bag:
            if token not in pars_bag:
                pars_bag[token] = 1
            else:
                pars_bag[token] += 1
    pars_n = cursor.rowcount

print(pars_n, 'paragrafos lidos')
print(len(pars_bag), 'tokens')

1987212it [01:34, 20968.25it/s]

1987212 paragrafos lidos
115679 tokens


In [5]:
pubs_bag = {}
pubs_bag_count = {}
pub_bag = {}

with mysql.connector.connect(**sql_config) as connection:
    cursor = connection.cursor()
    
    cursor.execute('SELECT conteudo FROM publicacao')
        
    for conteudo, in tqdm(cursor):
        pub_bag = dict()

        tokens = tokenizador.findall(re.sub('\d', ' ', conteudo.lower()))
        for token in tokens:
            if token not in pub_bag:
                pub_bag[token] = 1
            else:
                pub_bag[token] += 1
            
        for token in pub_bag:
            if token not in pubs_bag:
                pubs_bag[token] = 1
                pubs_bag_count[token] = pub_bag[token]
            else:
                pubs_bag[token] += 1
                pubs_bag_count[token] += pub_bag[token]
    pubs_n = cursor.rowcount
        
print(pubs_n, 'paragrafos lidos')
print(len(pubs_bag), 'tokens')

6512it [00:01, 3766.36it/s]

6512 paragrafos lidos
22458 tokens


In [26]:
pubs_2gram_bag = {}
pub_2gram_bag = {}

with mysql.connector.connect(**sql_config) as connection:
    cursor = connection.cursor()
    
    cursor.execute('SELECT conteudo FROM publicacao')
    
    for conteudo, in tqdm(cursor):
        pub_2gram_bag = dict()

        tokens = tokenizador.findall(re.sub('\d', ' ', conteudo.lower()))
        
        token_anterior = ''
        for token in tokens:
            chave = token_anterior + '-' + token
            pub_2gram_bag[chave] = True
            token_anterior = token
            
        for chave in pub_2gram_bag:
            if chave not in pubs_2gram_bag:
                pubs_2gram_bag[chave] = 1
            else:
                pubs_2gram_bag[chave] += 1
        
    pubs_2gram_n = cursor.rowcount
        
print(pubs_2gram_n, 'paragrafos lidos')
print(len(pubs_2gram_bag), 'tokens')

pars_2gram_bag = {}
par_2gram_bag = {}

with mysql.connector.connect(**sql_config) as connection:
    cursor = connection.cursor()
    
    cursor.execute('SELECT conteudo FROM paragrafo')
    
    for conteudo, in tqdm(cursor):
        par_2gram_bag = dict()

        tokens = tokenizador.findall(re.sub('\d', ' ', conteudo.lower()))
        
        token_anterior = ''
        for token in tokens:
            chave = token_anterior + '-' + token
            par_2gram_bag[chave] = True
            token_anterior = token
            
        for chave in par_2gram_bag:
            if chave not in pars_2gram_bag:
                pars_2gram_bag[chave] = 1
            else:
                pars_2gram_bag[chave] += 1
        
    pars_2gram_n = cursor.rowcount
        
print(pars_2gram_n, 'paragrafos lidos')
print(len(pars_2gram_bag), 'tokens')

6512it [00:01, 3327.41it/s]


6512 paragrafos lidos
132238 tokens


1987212it [01:30, 22057.89it/s]

1987212 paragrafos lidos
1438605 tokens


In [3]:
pubs_1gram_bag = {}
pubs_1gram_bag_idf = {}

with mysql.connector.connect(**sql_config) as connection:
    cursor = connection.cursor()
    
    cursor.execute('SELECT conteudo FROM publicacao')
    
    pub_1gram_bag = {}
    for conteudo, in tqdm(cursor):
        pub_1gram_bag = monta_saco_ngram(conteudo, n = 1)
        atualiza_saco_corpus(pubs_1gram_bag, pub_1gram_bag)
        atualiza_idf_corpus(pubs_1gram_bag_idf, pub_1gram_bag)
    pubs_1gram_n = cursor.rowcount
        
print(pubs_1gram_n, 'paragrafos lidos')
print(len(pubs_1gram_bag), 'tokens')

with mysql.connector.connect(**sql_config) as connection:
    cursor = connection.cursor()
    descricao = 'publicacoes_1gram'
    cursor.execute('INSERT INTO saco_corpus(descricao) VALUES(%s)', (descricao,))
    cursor.execute('SELECT saco_id FROM saco_corpus WHERE descricao LIKE %s', (descricao,))
    saco_id, = cursor.fetchone()
    print(saco_id)
    cursor.fetchall()
    for key in tqdm(pubs_1gram_bag):
        cursor.execute('INSERT INTO saco_item(saco_id, chave, frequencia, idf) VALUES (%s, %s, %s, %s)',
                       (saco_id, key, pubs_1gram_bag[key], pubs_1gram_bag_idf[key]))
    cursor.close()
    connection.commit()

pars_1gram_bag = {}
pars_1gram_bag_idf = {}

with mysql.connector.connect(**sql_config) as connection:
    cursor = connection.cursor()
    
    cursor.execute('SELECT conteudo FROM paragrafo')
    
    par_1gram_bag = {}
    for conteudo, in tqdm(cursor):
        par_1gram_bag = monta_saco_ngram(conteudo, n = 1)
        atualiza_idf_corpus(pars_1gram_bag_idf, par_1gram_bag)
        atualiza_saco_corpus(pars_1gram_bag, par_1gram_bag)
        
    pars_1gram_n = cursor.rowcount
        
print(pars_1gram_n, 'paragrafos lidos')
print(len(pars_1gram_bag), 'tokens')

with mysql.connector.connect(**sql_config) as connection:
    cursor = connection.cursor()
    descricao = 'paragrafos_1gram'
    cursor.execute('INSERT INTO saco_corpus(descricao) VALUES(%s)', (descricao,))
    cursor.execute('SELECT saco_id FROM saco_corpus WHERE descricao LIKE %s', (descricao,))
    saco_id, = cursor.fetchone()
    print(saco_id)
    cursor.fetchall()
    for key in tqdm(pars_1gram_bag):
        cursor.execute('INSERT INTO saco_item(saco_id, chave, frequencia, idf) VALUES (%s, %s, %s, %s)',
                       (saco_id, key, pars_1gram_bag[key], pars_1gram_bag_idf[key]))
    cursor.close()
    connection.commit()

6593it [00:01, 3365.55it/s]


6593 paragrafos lidos
22583 tokens
99879499163762688


100%|██████████| 22583/22583 [00:08<00:00, 2518.68it/s]
2009957it [01:33, 21569.59it/s]


2009957 paragrafos lidos
116068 tokens
99879499163762689


100%|██████████| 116068/116068 [00:42<00:00, 2712.68it/s]


In [14]:
pars_2gram_bag

{'-poder': 1882,
 'poder-executivo': 4905,
 'executivo-clodoaldo': 578,
 'clodoaldo-armando': 4743,
 'armando-gazzetta': 4519,
 '-prefeito': 1728,
 'prefeito-municipal': 27643,
 '-seção': 7363,
 'seção-i': 1165,
 'i-gabinete': 941,
 'gabinete-do': 10782,
 'do-prefeito': 10129,
 '-maria': 3655,
 'maria-josé': 1563,
 'josé-majô': 771,
 'majô-jandreice': 808,
 'jandreice-chefe': 722,
 'chefe-de': 1737,
 'de-gabinete': 2303,
 '-decretos': 689,
 'decretos-municipais': 1193,
 'municipais-republicado': 53,
 'republicado-por': 269,
 'por-ter': 595,
 'ter-saído': 301,
 'saído-com': 291,
 'com-incorreções': 293,
 '-decreto': 2601,
 'decreto-nº': 10383,
 'nº-de': 36357,
 'de-de': 47882,
 'de-dezembro': 13158,
 'dezembro-de': 12671,
 'de-e': 9745,
 'e-doc': 3673,
 'doc-nº': 2578,
 'nº-dispõe': 125,
 'dispõe-sobre': 3778,
 'sobre-feriados': 25,
 'feriados-e': 122,
 'e-ponto': 52,
 'ponto-facultativos': 11,
 'facultativos-no': 28,
 'no-exercício': 1960,
 'exercício-de': 3939,
 'de-o': 2977,
 'o-pref

In [49]:
pubs_3gram_bag = {}

with mysql.connector.connect(**sql_config) as connection:
    cursor = connection.cursor()
    
    cursor.execute('SELECT conteudo FROM publicacao')
    
    pub_3gram_bag = {}
    for conteudo, in tqdm(cursor):
        pub_3gram_bag = monta_saco_ngram(conteudo, n = 3)
        atualiza_idf_corpus(pubs_3gram_bag, pub_3gram_bag)
    pubs_3gram_n = cursor.rowcount
        
print(pubs_3gram_n, 'paragrafos lidos')
print(len(pubs_3gram_bag), 'tokens')

pars_3gram_bag = {}

with mysql.connector.connect(**sql_config) as connection:
    cursor = connection.cursor()
    
    cursor.execute('SELECT conteudo FROM paragrafo')
    
    par_3gram_bag = {}
    for conteudo, in tqdm(cursor):
        par_3gram_bag = monta_saco_ngram(conteudo, n = 3)
        atualiza_idf_corpus(pars_3gram_bag, par_3gram_bag)
        
    pars_3gram_n = cursor.rowcount
        
print(pars_3gram_n, 'paragrafos lidos')
print(len(pars_3gram_bag), 'tokens')

6512it [00:02, 2244.41it/s]


6512 paragrafos lidos
237121 tokens


1987212it [02:04, 16014.79it/s]

1987212 paragrafos lidos
3403676 tokens


In [56]:
only_corpus = 0
both = 0
only_pubs = 0

print_count = 50

for key in pars_2gram_bag:
    if key not in pubs_2gram_bag:
        only_corpus += 1
        
        if print_count > 0:
            print(key)
            print_count -= 1
    else:
        both += 1

print()
print_count = 50
        
for key in pubs_2gram_bag:
    if key not in pars_2gram_bag:
        only_pubs += 1
        if print_count > 0:
            print(key)
            print_count -= 1
        
print(only_corpus)
print(both)
print(only_pubs)

-poder
poder-executivo
executivo-clodoaldo
-prefeito
-seção
seção-i
i-gabinete
-maria
-decretos
decretos-municipais
municipais-republicado
republicado-por
-decreto
e-doc
doc-nº
nº-dispõe
dispõe-sobre
sobre-feriados
feriados-e
e-ponto
ponto-facultativos
facultativos-no
no-exercício
no-uso
suas-atribuições
atribuições-legais
legais-conferidas
conferidas-pelo
pelo-art
lei-orgânica
orgânica-do
se-planejar
planejar-o
dos-órgãos
-exercício
exercício-considerando
que-tradicionalmente
tradicionalmente-a
administração-decreta
decreta-ponto
ponto-facultativo
facultativo-em
no-transcorrer
transcorrer-do
ano-existem
existem-feriados
feriados-municipais
como-feriados
feriados-nacionais
nacionais-a

artrodar-leia
r-vier
qtd-bl
bl-especificação
especificação-bloco
marca-trevo
trevo-valor
qtd-pct
pct-especificação
especificação-papel
marca-romitec
romitec-valor
adjudicação-notificamos
comercial-notificamos
totalr-lote
diario-do
patrimonial-item
postos-unidade
dias-preço
posto-total
dia-posto
vigilante

In [135]:
df_pars = pd.DataFrame(pars_2gram_bag.items(), columns = ['2gram', 'freq'])
df_pubs = pd.DataFrame(pubs_2gram_bag.items(), columns = ['2gram', 'freq'])

In [245]:
score_bag_pub = {}
for key in pubs_3gram_bag_idf:
    if key in pars_3gram_bag_idf:
        score_bag_pub[key] = pubs_3gram_bag_idf[key]/pubs_3gram_n

score_bag_par = {}
for key in pubs_3gram_bag_idf:
    if key in pars_3gram_bag_idf:
        score_bag_par[key] = pars_3gram_bag_idf[key]/pubs_3gram_n
        
scores = {}

with mysql.connector.connect(**sql_config) as connection:
    cursor = connection.cursor()
    
    cursor.execute('SELECT paragrafo_id, conteudo FROM paragrafo')
    for paragrafo_id, conteudo in tqdm(cursor):
        score_pub = 0
        score_par = 0
        
        saco_par = monta_saco_ngram(conteudo, n = 3)
        
        for chave in saco_par:
            if chave in pubs_3gram_bag_idf:
                score_pub += saco_par[chave] * score_bag_pub[chave]
                score_par += saco_par[chave] * score_bag_par[chave]
        scores[paragrafo_id] = score_pub/(score_par + 1)

1987212it [02:05, 15874.33it/s]


In [247]:
df_scores = pd.DataFrame(scores.items(), columns = ['paragrafo_id', 'score'])
df_scores.nlargest(n = 20, columns = 'score')

good_scoring = df_scores[df_scores['score'] > 0.3]
samples = good_scoring.sample(n = 10)

In [256]:
cosseno_paragrafos = {}
jaccard_paragrafos = {}
dissimilaridade_paragrafos = {}

for _, paragrafo_id, score in samples.itertuples():
    print(paragrafo_id, score)
    
    with mysql.connector.connect(**sql_config) as connection:
        cursor = connection.cursor()
        
        cursor.execute('SELECT conteudo FROM paragrafo WHERE paragrafo_id = %s', (paragrafo_id,))
        conteudo, = cursor.fetchone()
        
        saco_alvo = monta_saco_ngram(conteudo, n = 2)

        cursor.fetchall()
        
        cosseno_publicacoes = {}
        jaccard_publicacoes = {}
        dissimilaridade_publicacoes = {}
        
        cursor.execute('SELECT publicacao_id, conteudo FROM publicacao')
        for publicacao_id, conteudo_publicacao in tqdm(cursor):
            saco_pub = monta_saco_ngram(conteudo_publicacao, n = 2)
            
            similaridade_cosseno = calcula_cosseno_sacos(saco_pub, saco_alvo)
            similaridade_jaccard = calcula_jaccard_sacos(saco_pub, saco_alvo)
            dissimilaridade_paragrafo = calcula_dissimilaridade_strings(conteudo, conteudo_publicacao)
            
            if similaridade_cosseno > 0.75:
                cosseno_publicacoes[publicacao_id] = similaridade_cosseno
            if similaridade_jaccard > 0.5:
                jaccard_publicacoes[publicacao_id] = similaridade_jaccard
            if dissimilaridade_paragrafo < 0.1:
                dissimilaridade_publicacoes[publicacao_id] = dissimilaridade_paragrafo
                
        cosseno_paragrafos[paragrafo_id] = cosseno_publicacoes
        jaccard_paragrafos[paragrafo_id] = jaccard_publicacoes
        dissimilaridade_paragrafos[paragrafo_id] = dissimilaridade_publicacoes
        
        cursor.close()

99870959696933458 0.3398593893203408


6512it [00:06, 1058.11it/s]


99870959695241855 0.3193445100014396


6512it [00:09, 716.43it/s]


99870959696533385 0.3233391152181252


...

6512it [00:05, 1097.23it/s]


99870959696524559 0.300644285159998


6512it [00:09, 666.76it/s]


99870959696019015 0.31377348176125586


6512it [00:05, 1091.92it/s]


In [257]:
df_cosseno = pd.DataFrame(cosseno_paragrafos.items(), columns = ['paragrafo_id', 'sim_cosseno'])
df_jaccard = pd.DataFrame(jaccard_paragrafos.items(), columns = ['paragrafo_id', 'sim_jaccard'])
df_dissims = pd.DataFrame(dissimilaridade_paragrafos.items(), columns = ['paragrafo_id', 'dissim_strings'])

In [258]:
df_dissims.merge(df_jaccard.merge(df_cosseno, on = 'paragrafo_id'), on = 'paragrafo_id')

,paragrafo_id,dissim_strings,sim_jaccard,sim_cosseno
0,99870959696933458,"{99870959695043080: 0.004846938775510204, 9987...","{99870959695043080: 0.6101694915254238, 998709...",{99870959695043080: 0.7809401184071024}
1,99870959695241855,{},{},"{99870959695036962: 0.8194773498796744, 998709..."
2,99870959696533385,"{99870959695037449: 0.002765663227316243, 9987...","{99870959695036788: 0.5647668393782384, 998709...","{99870959695037269: 0.7667159338030225, 998709..."
3,99870959695262495,{},{},{}
4,99870959695064085,{},{},{}
5,99870959695978717,{},{},{}
6,99870959696226151,{},{},{}
7,99870959696020157,{},{},{}
8,99870959696524559,{99870959695037069: 0.01365661554891785},{99870959695037069: 0.6003717472118959},{99870959695037069: 0.7985655713589018}
9,99870959696019015,{99870959695040530: 0.003708923670350864},{99870959695040530: 0.5696969696969697},{99870959695040530: 0.7677911161946466}


In [263]:
for paragrafo in dissimilaridade_paragrafos:
    similaridade_publicacao = dissimilaridade_paragrafos[paragrafo]
    max_sim = 1
    pub_id = 0
    
    for publicacao in similaridade_publicacao:
        similaridade = similaridade_publicacao[publicacao]
        if similaridade < max_sim:
            max_sim = similaridade
            pub_id = publicacao
    
    print('########################')
    print(max_sim)
    with mysql.connector.connect(**sql_config) as connection:
        cursor = connection.cursor()
        
        cursor.execute('SELECT conteudo FROM paragrafo WHERE paragrafo_id = %s', (paragrafo,))
        resultado, = cursor.fetchone()
        cursor.fetchall()
        
        if max_sim < 1:
            cursor.execute('SELECT conteudo, identificador FROM publicacao WHERE publicacao_id = %s', (pub_id,))
            resultado_, identificador = cursor.fetchone()
            print(paragrafo, pub_id)
            print(resultado)
            print('----------------------------')
            print(resultado_)
            cursor.fetchall()
        
        cursor.close()
        
        print(identificador)
        

########################
0.004846938775510204
99870959696933458 99870959695043080
NOTIFICAÇÃO DE ADJUDICAÇÃO - Edital nº. 030/2020 - Processo n.º 33.356/2015 – Modalidade: 
Concorrência Pública nº 001/2020 - Objeto: PERMISSÃO DE USO DE BEM PÚBLICO PARA A 
EXPLORAÇÃO DE UM PONTO PARA CANTINA NO CENTRO CULTURAL “CARLOS 
FERNANDES DE PAIVA” - Interessado: Secretaria Municipal de Cultura.  Notificamos aos interessados 
no processo licitatório epigrafado que o julgamento e a classificação havidos foram devidamente 
Homologados em 23/03/2020 e seu objeto Adjudicado em 08/04/2020, ambos pelo Prefeito Municipal 
à pessoa física CAMILA TRAVALINO ROBIN com a proposta DE ALUGUEL MENSAL: R$ 570,00 
(quinhentos e setenta reais) acrescido de 20% sobre o valor do aluguel, ao uso de água/esgoto e 
energia elétrica do Centro Cultural, totalizando R$ 684,00 (seiscentos e oitenta e quatro reais).
Bauru, 15/04/2020 – Talita Cristina Pereira Vicente – Diretora da Divisão de Licitações.

-------------------

In [155]:
df_pars = pd.DataFrame(pars_3gram_bag.items(), columns = ['ngram', 'freq'])
df_pubs = pd.DataFrame(pubs_3gram_bag.items(), columns = ['ngram', 'freq'])

In [164]:
df_ngrams = df_pars.merge(df_pubs, on = 'ngram', suffixes = ('_par', '_pub'))
df_ngrams['freq_par'] = df_ngrams['freq_par'].apply(lambda f: f/pars_3gram_n)
df_ngrams['freq_pub'] = df_ngrams['freq_pub'].apply(lambda f: f/pubs_3gram_n)

In [173]:
df_ngrams['diff'] = df_ngrams['freq_pub'] - df_ngrams['freq_par']

In [181]:
df_ngrams.nlargest(n = 50, columns = 'diff')

,ngram,freq_par,freq_pub,diff
1505,--notificação,0.007590,0.710534,0.702945
1506,-notificação-de,0.005225,0.707617,0.702391
1523,tipo-menor-preço,0.003962,0.605190,0.601229
308,modalidade-pregão-eletrônico,0.008062,0.560811,0.552749
1524,menor-preço-por,0.003602,0.524263,0.520661
348,bauru-sp-gov,0.016992,0.535780,0.518788
349,sp-gov-br,0.022289,0.536701,0.514412
334,compras-e-licitações,0.004722,0.500154,0.495432
325,prefeitura-municipal-de,0.018094,0.512439,0.494345
17,municipal-de-bauru,0.023370,0.515971,0.492601


In [182]:
df_ngrams.nsmallest(n = 50, columns = 'diff')

,ngram,freq_par,freq_pub,diff
1259,--processo,0.009715,0.000307,-0.009408
65,lei-municipal-nº,0.009030,0.000461,-0.008570
2024,--item,0.013603,0.005682,-0.007921
11,nº-de-de,0.010476,0.003225,-0.007251
22,de-bauru-e,0.006999,0.001382,-0.005617
2000,--lote,0.006708,0.001229,-0.005480
64,da-lei-municipal,0.005925,0.000461,-0.005464
276,processo-administrativo-nº,0.006173,0.001229,-0.004944
1581,ata-de-registro,0.009354,0.004760,-0.004594
3177,processo-nº-pregão,0.004720,0.000307,-0.004413
